In [26]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [36]:
product_names=['Redmi 9A (Nature Green, 2GB RAM, 32GB Storage)','OnePlus 9 5G (Arctic Sky, 8GB RAM, 128GB Storage)','Samsung Galaxy M51 (Electric Blue, 6GB RAM, 128GB Storage)','Oppo A31 (Fantasy White, 6GB RAM, 128GB Storage)','New Apple iPhone 12 (64GB) - (Product) RED','realme 8 5G (4 GB RAM, 64GB ROM)','Vivo Y51A (Titanium Sapphire, 8GB, 128GB Storage)']
asins=[]
for name in product_names:
    sess = HTMLSession()
    url=f'https://www.amazon.in/s?k={name}'
    info=sess.get(url)
    soup=BeautifulSoup(info.text,'html.parser')
    gets=soup.find_all('div',{'class':'a-section a-spacing-none'})
    for one in gets:
        try:
            one_name=one.find('a',{'class':'a-link-normal a-text-normal'})
            names=one_name.find('span').text
            if names.find(name)!=-1:
                two=one_name['href']
                asins.append(two.split('/')[3])
                break
        except:
            pass

In [37]:
asins

['B08696XB4B',
 'B089MV1ZSM',
 'B085J1J32G',
 'B08444SXZ6',
 'B08L5TGWD1',
 'B094ZP6TDH',
 'B08LRDP2Q6']

In [38]:
reviewlist = []

In [39]:
def wholedata(url):
    info = requests.get(url)
    soup = BeautifulSoup(info.text, 'html.parser')
    return soup

def reviewsf(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews: 
            onereview = {
            'Name': soup.title.text.replace('Amazon.in:Customer reviews:', ''),
            'Title': item.find('a', {'data-hook': 'review-title'}).text.strip(),
            'Rating': item.find('i', {'data-hook': 'review-star-rating'}).text,
            'Review': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            }
            reviewlist.append(onereview)                               #AttributeError: 'NoneType' object has no attribute 'text' so used try except block
    except:
        pass
y=1
for model in asins:    
    for x in range(1,10):
        soup = wholedata(f'https://www.amazon.in/product-reviews/{model}/ref=cm_cr_arp_d_paging_btm_next_2?pageNumber={x}')
        reviewsf(soup)
        next_page=soup.find('li', {'class': 'a-disabled a-last'})
        if not next_page:
            pass
        else:
            break
    print(f'Done with phone: {model}')
    y=y+1

Done with phone: B08696XB4B
Done with phone: B089MV1ZSM
Done with phone: B085J1J32G
Done with phone: B08444SXZ6
Done with phone: B08L5TGWD1
Done with phone: B094ZP6TDH
Done with phone: B08LRDP2Q6


In [40]:
reviewlist

[{'Name': ' Redmi 9A (Nature Green, 2GB RAM, 32GB Storage) | 2GHz Octa-core Helio G25 Processor | 5000 mAh Battery',
  'Title': "Don't buy this phone",
  'Rating': '1.0 out of 5 stars',
  'Review': "Both 13 MP and 5MP Camera quality is good. Have lot of pre installed apps. But can be uninstalled if not required. Overall good performance . I am using Asus zenfone max and according to me far better than Asus except Camera quality. It comes in 5000 mah battery and one can't get a better option at this price range. Go for it. It's really goodAfter one week useEdit:- please don't buy this phone. It hangs a lot and sometimes there is problem in touch screen."},
 {'Name': ' Redmi 9A (Nature Green, 2GB RAM, 32GB Storage) | 2GHz Octa-core Helio G25 Processor | 5000 mAh Battery',
  'Title': 'One of the good choice for low budget phone',
  'Rating': '4.0 out of 5 stars',
  'Review': "If you are looking for low budget phone this can be a good choice for you.If you wanna android phone just for day 

In [41]:
df = pd.DataFrame(reviewlist)
df.to_csv('Amazon_review_by_name.csv')